In [27]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import os

# 设置数据路径
load_path = "saved_scans/"  # 修改为你的实际路径
files = sorted([f for f in os.listdir(load_path) if f.endswith(".npz")])

# 设置 colormap
colormap = plt.get_cmap('jet')

# 创建坐标轴
axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0)

# 创建极坐标圆
def create_polar_rings(radii, z_height=0.0, resolution=100):
    rings = []
    for r in radii:
        theta = np.linspace(0, 2 * np.pi, resolution)
        x = r * np.cos(theta)
        y = r * np.sin(theta)
        z = np.full_like(x, z_height)
        pts = np.stack([x, y, z], axis=1)
        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector(pts)
        lines = [[i, (i+1)%resolution] for i in range(resolution)]
        line_set.lines = o3d.utility.Vector2iVector(lines)
        colors = [[0.5, 0.5, 0.5] for _ in lines]
        line_set.colors = o3d.utility.Vector3dVector(colors)
        rings.append(line_set)
    return rings

polar_rings = create_polar_rings([1.0, 2.0, 3.0])  # 半径可自调


for file in files:
    # 读取数据
    data = np.load(os.path.join(load_path, file))
    xyz = data['xyz']
    intensity = data['intensity']
    
    # 归一化 intensity
    intensity_norm = (intensity - np.min(intensity)) / (np.max(intensity) - np.min(intensity) + 1e-6)
    colors = colormap(intensity_norm)[:, :3]  # 去掉 alpha 通道

    # 创建 Open3D 点云对象
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    pcd.colors = o3d.utility.Vector3dVector(colors)

    

    # 显示点云
    print(f"Showing: {file}")
    o3d.visualization.draw_geometries([pcd, axis] + polar_rings, window_name=file)

    # 可选：按键跳过或结束
    user_input = input("Press Enter to see next, or type 'q' to quit: ")
    if user_input.lower() == 'q':
        break


Showing: 0_points_raw_1749546707.npz
Showing: 0_points_raw_1749546735.npz
Showing: 0_points_raw_1749546776.npz
Showing: 0_points_raw_1749546812.npz
Showing: 45_points_raw_1749547137.npz
Showing: 45_points_raw_1749547197.npz
Showing: 45_points_raw_1749547242.npz
Showing: 45_points_raw_1749547297.npz
Showing: pm_points_raw_1749545740.npz
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 


In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import os

# 设置数据路径
load_path = "saved_scans/"
files = sorted([f for f in os.listdir(load_path) if f.endswith(".npz")])

# 设置 colormap
colormap = plt.get_cmap('jet')

# 创建坐标轴
axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.01)

# 创建极坐标圆
def create_polar_rings(radii, z_height=0.0, resolution=100):
    rings = []
    for r in radii:
        theta = np.linspace(0, 2 * np.pi, resolution)
        x = r * np.cos(theta)
        y = r * np.sin(theta)
        z = np.full_like(x, z_height)
        pts = np.stack([x, y, z], axis=1)
        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector(pts)
        lines = [[i, (i+1)%resolution] for i in range(resolution)]
        line_set.lines = o3d.utility.Vector2iVector(lines)
        colors = [[0.5, 0.5, 0.5] for _ in lines]
        line_set.colors = o3d.utility.Vector3dVector(colors)
        rings.append(line_set)
    return rings

polar_rings = create_polar_rings([0.5, 1.0, 1.5, 2.0, 3.0])  # 半径列表

# 遍历所有文件
for file in files:
    print(f"Loading: {file}")
    data = np.load(os.path.join(load_path, file))
    
    # 加载字段
    xyz = data['xyz']
    intensity = data['intensity']
    r = data['r']  # 必须含有 r 字段

    # 筛选极径在 0.5 ~ 1.5 米之间的点
    mask_r = (r >= 0.4) & (r <= 1.3)
    xyz = xyz[mask_r]
    intensity = intensity[mask_r]

    # 归一化 intensity
    if len(intensity) == 0:
        print("No points in valid range, skipping...")
        continue
    intensity_norm = (intensity - np.min(intensity)) / (np.max(intensity) - np.min(intensity) + 1e-6)
    colors = colormap(intensity_norm)[:, :3]

    # 构造点云对象
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    pcd.colors = o3d.utility.Vector3dVector(colors)

    # 显示点云、坐标轴和极坐标环
    print(f"Showing: {file}, Points: {len(xyz)}")
    o3d.visualization.draw_geometries([pcd, axis] + polar_rings, window_name=file)

    # 用户操作
    user_input = input("Press Enter to see next, or type 'q' to quit: ")
    if user_input.lower() == 'q':
        break


Loading: pm_points_raw_1749545740.npz


KeyError: 'xyz is not a file in the archive'

In [25]:
import numpy as np
import open3d as o3d
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import os, gc

# 设置路径
load_path = "saved_scans/"
filename = "45_points_raw_1749547297.npz"
filepath = os.path.join(load_path, filename)

# 聚类参数
#DBSCAN_EPS = 0.025         #平面
#DBSCAN_EPS = 0.021         #0度
DBSCAN_EPS = 0.02          #45度

MIN_SAMPLES = 100
Z_SCALE = 0.2

# 极径筛选范围
#R_MIN = 0.4
#R_MAX = 1.4

R_MIN = 0.4
R_MAX = 0.9

# 加载数据
data = np.load(filepath)

# 筛选
mask_r = (data['r'] >= R_MIN) & (data['r'] <= R_MAX)

xyz_full = data['xyz'][mask_r].astype(np.float32)
intensity_full = data['intensity'][mask_r].astype(np.float32)
r = data['r'][mask_r].astype(np.float32)
ts = data['ts'][mask_r]
ch = data['ch'][mask_r]
theta = data['theta'][mask_r]
gc.collect()

# z缩放后的xyz，用于聚类
xyz_scaled = xyz_full.copy()
xyz_scaled[:, 2] *= Z_SCALE

# DBSCAN聚类
db = DBSCAN(eps=DBSCAN_EPS, min_samples=MIN_SAMPLES).fit(xyz_scaled)
labels = db.labels_
unique_labels = set(labels)
print(f"簇数量（不含噪声）: {len(unique_labels) - (1 if -1 in labels else 0)}")
print(f"噪声点数量: {np.sum(labels == -1)}")

# 给每个簇生成颜色，噪声点单独黑色
n_clusters = len(unique_labels) - (1 if -1 in unique_labels else 0)
cmap = plt.get_cmap('tab20')  # 20色循环使用
colors = np.zeros((len(xyz_full), 3), dtype=np.float32)

for label in unique_labels:
    if label == -1:
        # 噪声点黑色
        colors[labels == label] = [0, 0, 0]
    else:
        color = cmap(label % 20)[:3]  # 获取RGB，忽略alpha
        colors[labels == label] = color

# 构造点云
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_full)
pcd.colors = o3d.utility.Vector3dVector(colors)

# 生成坐标轴辅助线（长度1米）
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])

# 显示
print(f"显示点数：{len(xyz_full)}")
o3d.visualization.draw_geometries([pcd, axes], window_name="DBSCAN聚类结果（不同颜色为不同簇，黑色为噪声）")



簇数量（不含噪声）: 1
噪声点数量: 305
显示点数：4531


In [26]:
# 保存最大两个簇
save_dir = "clusterdata"
os.makedirs(save_dir, exist_ok=True)
save_file = os.path.join(save_dir, filename)

num_clusters_to_save = 1

# 过滤掉噪声点
unique_labels_arr, counts = np.unique(labels[labels != -1], return_counts=True)

if len(unique_labels_arr) < num_clusters_to_save:
    print(f"聚类结果不足 {num_clusters_to_save} 个簇，实际为 {len(unique_labels_arr)} 个。")
else:
    top_indices = np.argsort(counts)[-num_clusters_to_save:][::-1]
    top_labels = unique_labels_arr[top_indices]

    print(f"将保存的 {num_clusters_to_save} 个最大簇标签：{top_labels}")
    print(f"对应点数：{counts[top_indices]}")

    save_dict = {}
    for i, label in enumerate(top_labels, start=1):
        mask = (labels == label)
        prefix = f"cluster{i}"
        save_dict[f"{prefix}_xyz"] = xyz_full[mask]
        save_dict[f"{prefix}_intensity"] = intensity_full[mask]
        save_dict[f"{prefix}_ts"] = ts[mask]
        save_dict[f"{prefix}_ch"] = ch[mask]
        save_dict[f"{prefix}_r"] = r[mask]
        save_dict[f"{prefix}_theta"] = theta[mask]

    np.savez(save_file, **save_dict)
    print(f"已保存前 {num_clusters_to_save} 个最大类的数据到：{save_file}")

将保存的 1 个最大簇标签：[0]
对应点数：[4226]
已保存前 1 个最大类的数据到：clusterdata\45_points_raw_1749547297.npz


In [28]:
import numpy as np
import open3d as o3d
import os

# 读取文件
load_path = "clusterdata/"
filename = "45_points_raw_1749547197.npz"
filepath = os.path.join(load_path, filename)

# 加载数据
data = np.load(filepath)

# 构造 Open3D 点云列表
pcds = []
colors = [[1, 0, 0], [0, 1, 0]]  # 红色和绿色表示两个类

for i in [1, 2]:
    key_xyz = f"cluster{i}_xyz"
    if key_xyz not in data:
        print(f"未找到 {key_xyz}，跳过。")
        continue

    xyz = data[key_xyz]
    color = np.tile(colors[i - 1], (xyz.shape[0], 1))

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    pcd.colors = o3d.utility.Vector3dVector(color)
    pcds.append(pcd)

# 添加坐标轴辅助线
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])

# 可视化
print(f"展示两个最大簇，共加载 {len(pcds)} 个点云对象")
o3d.visualization.draw_geometries(pcds + [axes], window_name="两个最大聚类簇（红/绿）")


未找到 cluster2_xyz，跳过。
展示两个最大簇，共加载 1 个点云对象


In [1]:
import numpy as np
import open3d as o3d
import os
import time

# === 参数设置 ===
load_path = "clusterdata/"
colors = [[1, 0, 0], [0, 1, 0]]  # 红色、绿色分别表示 cluster1 和 cluster2

# === 可视化函数 ===
def visualize_clusters(npz_path):
    data = np.load(npz_path)

    pcds = []
    for i in [1, 2]:
        key_xyz = f"cluster{i}_xyz"
        if key_xyz not in data:
            print(f"文件 {os.path.basename(npz_path)} 缺少 {key_xyz}，跳过该簇。")
            continue

        xyz = data[key_xyz]
        color = np.tile(colors[i - 1], (xyz.shape[0], 1))

        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(xyz)
        pcd.colors = o3d.utility.Vector3dVector(color)
        pcds.append(pcd)

    if pcds:
        axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
        print(f"展示文件 {os.path.basename(npz_path)}，共 {len(pcds)} 个聚类簇点云")
        o3d.visualization.draw_geometries(pcds + [axes], window_name=os.path.basename(npz_path))
    else:
        print(f"文件 {os.path.basename(npz_path)} 中没有任何聚类簇数据可视化。")

# === 遍历所有 .npz 文件 ===
files = sorted([f for f in os.listdir(load_path) if f.endswith(".npz")])
if not files:
    print("文件夹中没有找到任何 .npz 文件。")
else:
    for f in files:
        filepath = os.path.join(load_path, f)
        visualize_clusters(filepath)
        time.sleep(0.5)  # 可选：避免快速切换窗口时卡顿


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
文件 0_points_raw_1749546707.npz 缺少 cluster2_xyz，跳过该簇。
展示文件 0_points_raw_1749546707.npz，共 1 个聚类簇点云
文件 0_points_raw_1749546735.npz 缺少 cluster2_xyz，跳过该簇。
展示文件 0_points_raw_1749546735.npz，共 1 个聚类簇点云
文件 0_points_raw_1749546776.npz 缺少 cluster2_xyz，跳过该簇。
展示文件 0_points_raw_1749546776.npz，共 1 个聚类簇点云
文件 0_points_raw_1749546812.npz 缺少 cluster2_xyz，跳过该簇。
展示文件 0_points_raw_1749546812.npz，共 1 个聚类簇点云
文件 45_points_raw_1749547137.npz 缺少 cluster2_xyz，跳过该簇。
展示文件 45_points_raw_1749547137.npz，共 1 个聚类簇点云
文件 45_points_raw_1749547197.npz 缺少 cluster2_xyz，跳过该簇。
展示文件 45_points_raw_1749547197.npz，共 1 个聚类簇点云
文件 45_points_raw_1749547242.npz 缺少 cluster2_xyz，跳过该簇。
展示文件 45_points_raw_1749547242.npz，共 1 个聚类簇点云
文件 45_points_raw_1749547297.npz 缺少 cluster2_xyz，跳过该簇。
展示文件 45_points_raw_1749547297.npz，共 1 个聚类簇点云
展示文件 pm_points_raw_1749545740.npz，共 2 个聚类簇点云
展示

In [2]:
import numpy as np
import os
import re
import csv

# === 文件路径设置 ===
data_dir = 'clusterdata/filtered_points'
equation_files = [
    'pm_points_raw_1749545740_filtered_fit_equations.txt',
    'pm_points_raw_1749545784_filtered_fit_equations.txt',
    'pm_points_raw_1749545893_filtered_fit_equations.txt',
    'pm_points_raw_1749545953_filtered_fit_equations.txt',
]
equation_paths = [os.path.join(data_dir, f) for f in equation_files]

# === 读取点云 npz 文件 ===
npz_file = 'pm_points_raw_1749545953_filtered_points.npz'
npz_path = os.path.join(data_dir, npz_file)
data = np.load(npz_path)
xyz = data['xyz']
ch = data['ch']

# === 解析拟合结果 ===
laser_fits = {}
pattern = re.compile(r'Laser (\d+): y = ([\d\.\-\+eE]+)x \+ ([\d\.\-\+eE]+)')

for filepath in equation_paths:
    with open(filepath, 'r') as f:
        for line in f:
            match = pattern.match(line.strip())
            if match:
                lid = int(match.group(1))
                m = float(match.group(2))
                b = float(match.group(3))
                laser_fits.setdefault(lid, []).append((m, b))

# === 计算每个通道的平均直线 ===
avg_lines = {}
for lid, fits in laser_fits.items():
    ms, bs = zip(*fits)
    m_avg = np.mean(ms)
    b_avg = np.mean(bs)
    avg_lines[lid] = (m_avg, b_avg)

# === 计算极角修正 & 极径 ===
results = {}
all_distances = []

for lid, (m_avg, b_avg) in avg_lines.items():
    dist = np.abs(b_avg) / np.sqrt(1 + m_avg**2)
    all_distances.append(dist)
    theta = np.arctan(-1 / m_avg)
    delta_theta_deg = np.degrees(theta)
    results[lid] = {
        'm_avg': m_avg,
        'b_avg': b_avg,
        'delta_theta_deg': delta_theta_deg,
        'distance': dist,
    }

# === 计算目标极径（平均） ===
target_radius = np.mean(all_distances)

# === 计算极径修正量 ===
for lid, r in results.items():
    r['delta_r'] = target_radius - r['distance']

print(f"目标极径（平均垂线长度）: {target_radius:.6f} m\n")
for lid in sorted(results.keys()):
    r = results[lid]
    print(f"Laser {lid}: Δθ = {r['delta_theta_deg']:+.6f}° , Δr = {r['delta_r']:+.6f} m")

# === 获取每个通道的一个 z 值 ===
lid_z_value = {}
for lid in sorted(results.keys()):
    mask = ch == lid
    if not np.any(mask):
        print(f"⚠️ 通道 {lid} 无点云，z 值设置为 NaN")
        z_val = np.nan
    else:
        z_val = xyz[mask, 2][0]  # 取第一个点的 z 值
    lid_z_value[lid] = z_val

# === 写入 CSV ===
output_path = os.path.join(data_dir, 'laser_corrections_avgline0.csv')
with open(output_path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['lid', 'delta_theta_deg', 'delta_r', 'z'])
    for lid in sorted(results.keys()):
        r = results[lid]
        z_val = lid_z_value[lid]
        writer.writerow([
            lid,
            f"{r['delta_theta_deg']:.6f}",
            f"{r['delta_r']:.6f}",
            f"{z_val:.6f}" if not np.isnan(z_val) else 'NaN'
        ])

print(f"\n修正量已保存到: {output_path}")


目标极径（平均垂线长度）: 0.807577 m

Laser 0: Δθ = -8.447909° , Δr = -0.001468 m
Laser 1: Δθ = -5.885325° , Δr = -0.008842 m
Laser 2: Δθ = -6.779101° , Δr = -0.002797 m
Laser 3: Δθ = -5.345246° , Δr = -0.043882 m
Laser 4: Δθ = -7.399571° , Δr = -0.006863 m
Laser 5: Δθ = -5.336076° , Δr = -0.021831 m
Laser 6: Δθ = -8.192075° , Δr = +0.002907 m
Laser 7: Δθ = -5.614298° , Δr = -0.020298 m
Laser 8: Δθ = -3.212760° , Δr = +0.024988 m
Laser 9: Δθ = -2.849118° , Δr = +0.013039 m
Laser 10: Δθ = -2.607934° , Δr = +0.023142 m
Laser 11: Δθ = -2.822845° , Δr = +0.010213 m
Laser 12: Δθ = -2.404432° , Δr = +0.012658 m
Laser 13: Δθ = -3.238048° , Δr = +0.006679 m
Laser 14: Δθ = -2.571400° , Δr = +0.011898 m
Laser 15: Δθ = -3.195812° , Δr = +0.000458 m

修正量已保存到: clusterdata/filtered_points\laser_corrections_avgline0.csv


In [103]:
import numpy as np
import open3d as o3d
import os
import matplotlib.pyplot as plt
import csv

# === 参数设置 ===
data_dir = 'clusterdata'
npz_file = 'pm_points_raw_1749545953.npz'
npz_path = os.path.join(data_dir, npz_file)

correction_file = os.path.join(data_dir, 'filtered_points/laser_corrections_avgline0.csv')

# === 读取修正量 ===
angle_corrections = {}
radius_corrections = {}
base_z_corrections = {}

with open(correction_file, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        lid = int(row['lid'])
        delta_theta = np.radians(float(row['delta_theta_deg']))
        delta_r = float(row['delta_r'])
        base_z = float(row['z'])
        angle_corrections[lid] = delta_theta
        radius_corrections[lid] = delta_r
        base_z_corrections[lid] = base_z


# === 读取 npz 文件 ===
def load_cluster_npz(npz_path):
    data = np.load(npz_path)
    clusters = [key for key in data.keys() if key.endswith("_xyz")]
    if not clusters:
        raise ValueError(f"文件 {npz_path} 中未找到 cluster*_xyz 数据")
    
    all_xyz = []
    all_ch = []
    all_r = []
    all_theta = []
    
    for cluster_xyz_key in clusters:
        prefix = cluster_xyz_key.replace("_xyz", "")
        xyz = data[cluster_xyz_key]
        ch = data.get(f"{prefix}_ch")
        r = data.get(f"{prefix}_r")
        theta = data.get(f"{prefix}_theta")
        
        if ch is None or r is None or theta is None:
            raise ValueError(f"{prefix} 缺少 ch/r/theta 数据")
        
        all_xyz.append(xyz)
        all_ch.append(ch)
        all_r.append(r)
        all_theta.append(theta)
    
    xyz = np.vstack(all_xyz)
    ch = np.concatenate(all_ch)
    r = np.concatenate(all_r)
    theta = np.concatenate(all_theta)
    
    return xyz, ch, r, theta

xyz, ch, r, theta = load_cluster_npz(npz_path)

# === 应用极角与极径修正 ===
xyz_corrected = xyz.copy()
base_radius = 0.807577  #delta_r的来源平面距离激光雷达的距离

def raised_cosine_peak(z, z0=-0.11, A=0.8, width=0.08):
    delta = np.abs(z - z0)
    peak = np.zeros_like(z)
    mask = delta <= width
    peak[mask] = A * 0.5 * (1 + np.cos(np.pi * (z[mask] - z0) / width))
    return peak

def smooth_peak(z, z0=-0.08, A=0.04, sigma=0.04):
    return A * np.exp(-((z - z0) ** 2) / (2 * sigma ** 2))


for lid in np.unique(ch):
    mask = ch == lid
    delta_theta = angle_corrections.get(lid, 0.0)
    delta_r = radius_corrections.get(lid, 0.0)
    this_base_z = base_z_corrections.get(lid, 0.0)

    x = xyz[mask, 0]
    y = xyz[mask, 1]
    z = xyz[mask, 2]

    r_xy = np.sqrt(x**2 + y**2)
    #factor = np.sqrt(r_xy**2 + z**2) / np.sqrt(base_radius**2 + base_z**2)
    #factor = r_xy / base_radius

    r_xy += delta_r # + 8 * (2*z - 0.3) * (z + 0.2) * (z)  + 0.2 * z - smooth_peak(z, z0 = -0.085, A = 0.05, sigma=0.03) -smooth_peak(z, z0 = -0.18, A = 0.07, sigma=0.04)#raised_cosine_peak(z)# * abs(factor) # + 10 * (z - 0.2) * (z + 0.1) * z #+ 90 * z**2*(z+0.15)*(z-0.1)

    angles = np.arctan2(y, x) - delta_theta

    x_corr = r_xy * np.cos(angles)
    y_corr = r_xy * np.sin(angles)

    xyz_corrected[mask, 0] = x_corr
    xyz_corrected[mask, 1] = y_corr
    # z 不变


# === 构造点云对象 ===
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_corrected)

# === 可视化颜色（z 方向归一化上色） ===
z_vals = xyz_corrected[:, 2]
z_min, z_max = np.min(z_vals), np.max(z_vals)
norm_z = (z_vals - z_min) / (z_max - z_min + 1e-8)
colors = plt.cm.viridis(norm_z)[:, :3]
pcd.colors = o3d.utility.Vector3dVector(colors)

# === 坐标轴 ===
axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)

# === 极坐标环 ===
polar_rings = []
ring_radii = np.linspace(0.5, 5.0, 10)
for radius in ring_radii:
    theta_vals = np.linspace(0, 2 * np.pi, 300)
    ring_x = radius * np.cos(theta_vals)
    ring_y = radius * np.sin(theta_vals)
    ring_z = np.zeros_like(ring_x)
    ring_pts = np.stack([ring_x, ring_y, ring_z], axis=-1)
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(ring_pts)
    lines = [[i, i+1] for i in range(len(ring_pts)-1)]
    lines.append([len(ring_pts)-1, 0])
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector([[0.5, 0.5, 0.5] for _ in lines])
    polar_rings.append(line_set)

# === 拟合直线显示（用于校验拟合效果） ===
#equation_file = os.path.join(data_dir, npz_file.replace('_points.npz', '_fit_equations.txt'))
#
#line_sets = []
#if os.path.exists(equation_file):
#    import re
#    pattern = re.compile(r'Laser (\d+): y = ([\d\.\-\+eE]+)x \+ ([\d\.\-\+eE]+)')
#    with open(equation_file, 'r') as f:
#        for line in f:
#            match = pattern.match(line.strip())
#            if match:
#                lid = int(match.group(1))
#                m = float(match.group(2))
#                b = float(match.group(3))
#
#                # 生成直线段
#                xs = np.linspace(-5, 5, 2)
#                ys = m * xs + b
#                zs = np.zeros_like(xs)
#                pts = np.stack([xs, ys, zs], axis=-1)
#
#                line = o3d.geometry.LineSet()
#                line.points = o3d.utility.Vector3dVector(pts)
#                line.lines = o3d.utility.Vector2iVector([[0,1]])
#                line.colors = o3d.utility.Vector3dVector([[1.0, 0.0, 0.0]])  # 红色
#                line_sets.append(line)

# === 显示 ===
print(f"显示: {os.path.basename(npz_file)}, 点数: {len(xyz_corrected)}")
o3d.visualization.draw_geometries([pcd, axis] + polar_rings, window_name=os.path.basename(npz_file))


显示: pm_points_raw_1749545953.npz, 点数: 13916


In [104]:
# === 导出修正后数据 ===
output_dir = os.path.join(data_dir, 'corrected_points')
os.makedirs(output_dir, exist_ok=True)

output_npz_file = os.path.splitext(npz_file)[0] + '_corrected.npz'
output_npz_path = os.path.join(output_dir, output_npz_file)

np.savez_compressed(output_npz_path, xyz=xyz_corrected, ch=ch)

print(f"修正后点云已保存: {output_npz_path}")


修正后点云已保存: clusterdata\corrected_points\pm_points_raw_1749545953_corrected.npz
